In [19]:
import time

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.algorithms import QSVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

Loading Datasets

In [2]:
# Direct Kernel Set Training Data
dks_train_1 = pd.read_csv('data/Direct_Kernel_Set_I_Training.csv')

# Direct Kernel Set Classifcation Data
dks_class_1 = pd.read_csv('data/Direct_Kernel_Set_I_Classifications.csv')

x1_train = dks_class_1.iloc[:, 1:3]
y1_train = dks_class_1.iloc[:, 3]

x1_test = dks_train_1.iloc[:, 1:3]
y1_test = dks_train_1.iloc[:, 3]

In [3]:
x2_train, y2_train, x2_test, y2_test = ad_hoc_data(training_size = 80, test_size = 20, n=2, gap = 0.2, one_hot = False)

In [66]:
(x3_train, y3_train), (x3_test, y3_test) = keras.datasets.mnist.load_data()
x3_train = x3_train.reshape((60000, 28*28))
x3_test = x3_test.reshape((-1, 28*28))
x3_train.shape
num_dims = 10
#idx = list(range(0, 28*28, int(28*28 / num_dims) + 1))
#x3_train[:10, idx]

def limit_data(x, y, nums):
    idx = (y != y)
    for i in nums:
        idx = idx | (y == i)
    return x[idx], y[idx]

x3_train, y3_train = limit_data(x3_train, y3_train, [3, 6])
x3_test, y3_test = limit_data(x3_test, y3_test, [3, 6])
y3_train = (y3_train == 3).astype(int)
y3_test = (y3_test == 3).astype(int)

scaler = StandardScaler().fit(x3_train)
x3_train = scaler.transform(x3_train)
x3_test = scaler.transform(x3_test)

pca = PCA(n_components=num_dims).fit(x3_train)
x3_train = pca.transform(x3_train)
x3_test = pca.transform(x3_test)
print(pca.explained_variance_ratio_)

samples = np.append(x3_train, x3_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
x3_train = minmax_scale.transform(x3_train)
x3_test = minmax_scale.transform(x3_test)

[0.08939126 0.05811813 0.04367666 0.02961373 0.02748219 0.02472024
 0.01852863 0.01558034 0.01494015 0.01389285]


Training, Timing, and Evaluating Datasets

In [41]:
# First dataset (from paper)
random_seed = 10598
feature_map = ZZFeatureMap(feature_dimension=2,
                                 reps=2, entanglement='full')
backend = QuantumInstance(BasicAer.get_backend('statevector_simulator'), shots=1,
                                seed_simulator=random_seed, seed_transpiler=random_seed)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

qsvc = QSVC(quantum_kernel=kernel)
start_time = time.time()
qsvc.fit(x1_train, y1_train)#train
train_time = time.time() - start_time
qsvc_score = qsvc.score(x1_test, y1_test) #predict
test_time = time.time() - start_time - train_time
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(qsvc_score)
print(kernel.feature_map.num_qubits)

Training time:1.476001501083374[sec]
Test time:1.3499977588653564[sec]
1.0


In [55]:
# Second dataset (ad hoc data)
random_seed = 10598
feature_map = ZZFeatureMap(feature_dimension=2,
                                 reps=2, entanglement='full')
backend = QuantumInstance(BasicAer.get_backend('statevector_simulator'), shots=1,
                                seed_simulator=random_seed, seed_transpiler=random_seed)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

qsvc = QSVC(quantum_kernel=kernel)
start_time = time.time()
qsvc.fit(x2_train, y2_train)#train
train_time = time.time() - start_time
qsvc_score = qsvc.score(x2_test, y2_test) #predict
test_time = time.time() - start_time - train_time
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(qsvc_score)
print(kernel.feature_map.num_qubits)

Training time:2.312774896621704[sec]
Test time:0.4010016918182373[sec]
1.0
2


In [67]:
# Third dataset (mnist dataset)
random_seed = 10598
feature_map = ZZFeatureMap(feature_dimension=num_dims,
                                 reps=1, entanglement='full')
backend = QuantumInstance(BasicAer.get_backend('statevector_simulator'), shots=1,
                                seed_simulator=random_seed, seed_transpiler=random_seed)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

qsvc = QSVC(quantum_kernel=kernel)
start_time = time.time()
qsvc.fit(x3_train[:100], y3_train[:100])#train
train_time = time.time() - start_time
qsvc_score = qsvc.score(x3_test[:100], y3_test[:100]) #predict
test_time = time.time() - start_time - train_time
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(qsvc_score)
print(kernel.feature_map.num_qubits)

Training time:14.850369453430176[sec]
Test time:28.344998121261597[sec]
0.9
10
